In [1]:
# In Jupyter Notebook's kernel.json, set "env": {"PYTHONHASHSEED":"0"} for reproducability

In [2]:
import networkx as nx
import json
import csv
import math
import time
from random import seed, shuffle, randint, sample
from itertools import chain

# Load dataset

In [3]:
# SOURCE: https://github.com/villmow/datasets_knowledge_embedding/blob/master/FB15k-237/entity2wikidata.json
# with open('entity2wikidata.json', 'r') as f:
#     fb_dict = json.load(f)

%store -r fb_dict

In [4]:
def get_dict(txt):
    lines = txt.read().split("\n")
    new_dict = {}
    for line in lines:
        [ID, val] = line.split("\t")
        new_dict[ID] = val
    return new_dict

In [5]:
def load_data():
    if dataset == "Umls" or dataset == "Nations":
        entities = open(dataset + "/entities.txt","r") 
        relations = open(dataset + "/relations.txt", "r")
        triples = open(dataset + "/triples.txt", "r")

        e_dict = get_dict(entities)
        r_dict = get_dict(relations)
        triples = triples.read().split("\n")

    elif dataset == "FB15K":
        triples_temp = open("fb_train.txt", "r").read().split("\n")
        
        e_dict = fb_dict
        r_dict = []
        triples = []
        
        for t in triples_temp:
            [ent1, rel, ent2] = t.split("\t")
            if ent1 in e_dict and ent2 in e_dict:
                r_dict.append(rel)
                triples.append(t)
        
    return e_dict, r_dict, triples

# Algorithm Functions

In [6]:
# Takes in a KG, returns its NetworkX graph and 
# a dictionary where the key is an edge pair (obj1, obj2)
# and the value is the edge's label

def construct_DiGraph(KG):
    G = nx.DiGraph()
    G_labels = {}
    for t in KG:
        triple = t.split("\t")
        if dataset == "Nations" or dataset == "Umls":
            rel = triple[0]
            ent_1 = triple[1]
            ent_2 = triple[2]
        elif dataset == "FB15K":
            ent_1 = triple[0]
            rel = triple[1].split("/")[-1]
            ent_2 = triple[2]
        G_labels[(ent_1, ent_2)] = rel
        G.add_edge(ent_1, ent_2)
        
    return G, G_labels

In [7]:
# G_0 is the original graph represented as a set of the original tab separated string lines
# D and A are some threshold percentages (expressed as integers)
# L is the set of possible edge label id's
# SizeU is the desired size of U

def generate_U(G0, D, A, L, SizeU):
    
    # To ensure that there are no identical KGs
    U_set = set([frozenset(G0)])
    
    U = {}
    U[0] = G0
   
    while len(U) < SizeU:
        d = randint(0, D)
        a = randint(0, A)
        lst = list(U_set)
        shuffle(lst)
        G_prime = set(lst[0])
        G_prime_graph, _ = construct_DiGraph(G_prime)
        G_prime_size = len(G_prime)
        G_prime_nodes = G_prime_graph.nodes()
        
        num_edges_to_delete = int(round(d/100*G_prime_size))
        num_edges_to_add = int(round(a/100*G_prime_size))
        
        # delete d% of edges in G'
        lst = list(G_prime)
        shuffle(lst)
        G_prime = set(lst[:G_prime_size-num_edges_to_delete])
        
        # add a% of edges to G'
        for _ in range(num_edges_to_add):
            new_edge = sample(G_prime, 1)[0] # initialize to a known edge 
            while new_edge in G_prime:
                lst = list(L)
                shuffle(lst)
                new_label = lst[0]
                lst = list(e_dict)
                shuffle(lst)
                new_endpoints = lst[:2]
                if dataset == "Nations" or dataset == "Umls":
                    new_edge = str(new_label) + "\t" + str(new_endpoints[0]) + "\t" + str(new_endpoints[1])
                elif dataset == "FB15K":
                    new_edge = str(new_endpoints[0])+"\t"+"addedEdge/" + new_label + "\t" + str(new_endpoints[1])
                
            G_prime.add(new_edge)
            
        G_prime_size = len(G_prime)
        
        if frozenset(G_prime) not in U_set and G_prime_size >= 8 and G_prime_size <=12:
            U[len(U)] = G_prime
            U_set.add(frozenset(G_prime))
    
    return U

In [8]:
# returns distance graph GI of KGs

def generate_GI(U, d, t):
    G_I = nx.Graph()
    # Add edge if distance between KGs falls within range t
    for i in range(len(U)):
        for j in range(i+1, len(U)):
            dist = d(U[i], U[j])
            if dist <= t[1] and dist >= t[0]:
                G_I.add_edge(i, j)
    return G_I

In [9]:
def jacardian_dist(KG_a, KG_b): 
    KG_intersection = len(KG_a.intersection(KG_b))
    KG_union = len(KG_a.union(KG_b))
    if (KG_union == 0):
        return 1
    else:
        return 1-(KG_intersection/KG_union)

In [10]:
def clique_computation(G_I, K_0, n):
    G = G_I
    C = []
    C_nodes = set()
    C_graph = nx.DiGraph()

    while nx.graph_clique_number(G) > n:
        all_max_cliques = list(filter(lambda x: len(x) > n, list(nx.find_cliques(G))))
        shuffle(all_max_cliques)
        max_clique = set(all_max_cliques[0])
        
        # Construct max_clique_graph by finding the subgraph of G with max_clique's nodes
        max_clique_graph = G.subgraph(max_clique)
        
        # Add this max_clique to C
        C.append(max_clique)

        # Adds the nodes of max_clique to C_nodes
        C_nodes.update(max_clique)
        
        # Adds max_clique_graph to C_graph
        C_graph = nx.algorithms.operators.binary.compose(C_graph, max_clique_graph) 
        
        if K_0 in max_clique:
            return C
        
        G.remove_nodes_from(C_nodes)
        
    return C

In [11]:
def clique_fakeKG(G_I, K_0, n):
    C = clique_computation(G_I, K_0, n)
    
    shuffle(C)
    for clique in C:
        if K_0 in clique:
            clique.remove(K_0)
            lst = list(clique)
            shuffle(lst)
            K = set(lst[:n])
            K.add(K_0)
            return K
        
    return set()

# Generating Test Fakes

## Filter misleading data for Nations

In [12]:
def get_nation_data(triples):
    # ids of triples that are not misleading
    nation_rels = list(chain(range(0,12), range(162, 242), range(255, 283), \
                              range(590, 613),range(677, 691), range(703, 735)))

    nation_triples = list(map(lambda x: triples[x], nation_rels))

    # ids of relationships that are not misleading
    nation_rels = [0,5,6,8,27,33,35]
    return nation_triples, nation_rels

In [13]:
def generate_fakes(i, D, A, k, U_size):
    global e_dict, r_dict, triples, time_taken
    seed(i)              
    e_dict, r_dict, triples = load_data()

    if dataset == "Nations":
        nation_triples, nation_rels = get_nation_data(triples)
        G_0 = set()
        shuffle(nation_triples)
        G_0.update(nation_triples[:k])
        U = generate_U(G_0, D, A, nation_rels, U_size)
    elif dataset == "Umls":
        G_0 = set()
        shuffle(triples)
        G_0.update(triples[:k])
        U = generate_U(G_0, D, A, set(range(len(r_dict))), U_size)
    elif dataset == "FB15K":
        G_0 = set()
        shuffle(triples)
        G_0.update(triples[:k])
        U = generate_U(G_0, D, A, r_dict, U_size)
    
    d = jacardian_dist
    K_0 = 0
    n = 3

    t_intervals = [[0,.33],[0.33,0.66], [.66,1]]
    KGs = []
    intervals_dict = {}
    fails = []

    for t in t_intervals:
        G_I = generate_GI(U, d, t)
        result = clique_fakeKG(G_I, K_0, n)
        if len(result) == 0:
            fails.append("fail")
            break
        else:
            KGs.append(result)
            intervals_dict[str(t)] = result
            
    flat_KGs = list(set([item for KG in KGs for item in KG]))
    shuffle(flat_KGs)

    return U, flat_KGs, intervals_dict, fails

## Finding tests that contain 3 fake graphs per interval

In [14]:
def generate_tests(dataset):
    global results_txt
    U_size = 50

    all_tests = []
    for i in range(0,22):
        results_txt += "\nSeed: {}\n".format(i)
        print("i:", i)
        valid_tests = []
        for D in range(20,60,5):
            for A in range(20, 60, 5):
                for k in range(8,13):
                    U, flat_KGs, intervals_dict, fails = generate_fakes(i, D, A, k, U_size)
                    if len(fails) == 0:
                        valid_tests.append([i, D, A, k, U_size])
                        break

        results_txt += str(valid_tests) + "\n"
        shuffle(valid_tests)
        all_tests.append(valid_tests[0])
            
    return all_tests

## Run single test

In [15]:
def generate_summary(U, flat_KGs, intervals_dict, testnumber):
    summary_filename = "FakeKGTests/" + dataset + "/Test" + str(testnumber) + "/summary.txt"

    txt = "Dataset: {}\n".format(dataset)
    txt += "\nTest number: {}\n".format(testnumber)

    txt += "\nOrder of graphs in test:\n"
    for idx in range(len(flat_KGs)):
        txt += "Graph {}: KG {}\n".format(idx+1, flat_KGs[idx])

    summary_lines = [txt,
                     "\nParameters:\n", "i is {}, D is {}, A is {}, k is {}, U_size is {}\n"\
                              .format(i, D, A, k, U_size), "\nIntervals:\n"]
    for idx in intervals_dict:
        summary_lines.append(idx + ": " + str(intervals_dict[idx]) + "\n")

    for idx1 in range(len(flat_KGs)):
        txt = "\nPairwise distances for Graph {} (KG {}):\n".format(idx1+1, flat_KGs[idx1]) 
        for idx2 in range(len(flat_KGs)):
            txt += "from Graph {} (KG {}): {}\n".format(idx2+1, flat_KGs[idx2], \
                                                        jacardian_dist(U[flat_KGs[idx1]], U[flat_KGs[idx2]]))
        summary_lines.append(txt)

    summary_file = open(summary_filename,"w") 
    summary_file.writelines(summary_lines)
    summary_file.close()

In [16]:
def generate_JSON(U, flat_KGs, testnumber):
    for a in range(1, len(flat_KGs)+1):
        KG_id = flat_KGs[a-1]
        json_links = []
        json_nodes = []

        G, G_labels = construct_DiGraph(U[KG_id]) 
        for node in G.nodes():

            if dataset == "Nations" or dataset == "Umls":
                new_node = {
                    "name": e_dict[node],
                    "id": node
                }
            elif dataset == "FB15K":
                new_node = {
                    "name": e_dict[node]["label"],
                    "id": node
                }
            json_nodes.append(new_node)

        for edge in G.edges():
            if dataset == "Nations" or dataset == "Umls":
                new_link = {
                    "source": edge[0],
                    "target": edge[1],
                    "type": r_dict[G_labels[(edge[0], edge[1])]]
                }
            elif dataset == "FB15K":
                new_link = {
                    "source": edge[0],
                    "target": edge[1],
                    "type": G_labels[(edge[0], edge[1])].split("/")[-1]
                }
            json_links.append(new_link)

        json_dict = {
            "nodes": json_nodes,
            "links": json_links
        }

        graph_filename = "FakeKGTests/" + dataset + "/Test" + str(testnumber) + "/Graph" + str(a) + ".json" 

        with open(graph_filename, 'w') as json_file:
          json.dump(json_dict, json_file)

In [17]:
def generate_questions():
    questions_filename = "FakeKGTests/" + dataset + "/questions.txt"
    
    twentytwo_tests = all_tests
    shuffle(twentytwo_tests)
    five_tests = twentytwo_tests[:5]

    questions_lines = ""
    for test in five_tests:
        [i, D, A, k, U_size] = test
        U, flat_KGs, intervals_dict, fails = generate_fakes(i, D, A, k, U_size)
        original_graph = list(U[0])
        shuffle(original_graph)
        
        if dataset == "FB15K":
            [obj1, rel, obj2] = original_graph[0].split("\t")
            rel = rel.split("/")[-1]
            obj1 = e_dict[obj1]["label"]
            obj2 = e_dict[obj2]["label"]
        else:
            [rel, obj1, obj2] = original_graph[0].split("\t")
            rel = r_dict[rel]
            obj1 = e_dict[obj1]
            obj2 = e_dict[obj2]
                            
        questions_lines += obj1 + " and " + obj2 + " are related by " + rel + "\n"
        
        if dataset == "Nations":
            nation_triples, nation_rels = get_nation_data(triples)
            lst = list(nation_rels)
        else:
            lst = list(r_dict)
            
        shuffle(lst)
        answer_choices = set([rel])

        while len(answer_choices) < 5:
            new_choice = lst.pop()
            if dataset == "FB15K":
                answer_choices.add(new_choice.split("/")[-1])
            elif dataset == "Nations":
                answer_choices.add(r_dict[str(new_choice)])
            else:
                answer_choices.add(r_dict[new_choice])

        answer_choices = list(answer_choices)
        shuffle(answer_choices)
        for choice in answer_choices:
            questions_lines += choice + "\n"
            
        questions_lines += "\n"
        
    questions_file = open(questions_filename,"w") 
    questions_file.writelines(questions_lines)
    questions_file.close()

In [18]:
def generate_csv():
    csv_filename = "FakeKGTests/" + dataset + "summary.csv"  
    csv_lines = ""

    for test in all_tests:
        [i, D, A, k, U_size] = test
        U, flat_KGs, intervals_dict, fails = generate_fakes(i, D, A, k, U_size)

        for KG in flat_KGs:
            csv_lines += str(round(jacardian_dist(U[KG], U[0]), 2)) + ","
        csv_lines += "\n"
    
    csv_file = open(csv_filename,"w") 
    csv_file.writelines(csv_lines)
    csv_file.close()

In [19]:
dataset = "FB15K"

results_filename = "AllResults/" + dataset + "Results.txt"
results_file = open(results_filename,"w") 
results_txt = "Dataset: {}\n".format(dataset)

all_tests = generate_tests(dataset)

testnumber = 1
for test in all_tests:
    [i, D, A, k, U_size] = test
    U, flat_KGs, intervals_dict, fails = generate_fakes(i, D, A, k, U_size)
    generate_summary(U, flat_KGs, intervals_dict, testnumber)
    generate_JSON(U, flat_KGs, testnumber)
    testnumber += 1

results_file.writelines(results_txt)
results_file.close()
    
generate_questions()
generate_csv()

i: 0
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
i: 7
i: 8
i: 9
i: 10
i: 11
i: 12
i: 13
i: 14
i: 15
i: 16
i: 17
i: 18
i: 19
i: 20
i: 21
